In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
from tqdm import tqdm
import multiprocessing as mp
import spacy 
import en_core_web_sm
#import matplotlib.pyplot as plt
import nltk
nltk.download()
nltk.download('universal_tagset')
nltk.download('names')
nltk.download('brown')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from sklearn.feature_extraction.text import CountVectorizer
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter
from nltk.corpus import stopwords
from normalise import normalise
import string
import io
import re
from sklearn.utils import shuffle
import langid

In [ ]:
#get yelp dataset from kaggle
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
from google.colab import files
files.upload()
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d yelp-dataset/yelp-dataset
!ls
!unzip yelp-dataset.zip

In [ ]:
# convert json file to csv
data = {'stars': [], 'text': []}
n=0
with open('./yelp_academic_dataset_review.json') as f:
  for line in tqdm(f):
      review = json.loads(line)
      data['stars'].append(review['stars'])
      data['text'].append(review['text'])
      #data['useful'].append(review['useful'])
      #data['funny'].append(review['funny'])
      #data['cool'].append(review['cool'])
      #data['business_id'].append(review['business_id'])
      n = n+1

  df = pd.DataFrame(data)
  df.to_csv('yelp_academic_dataset_review.csv', sep='\t')


print(df.shape)
df.tail()

In [ ]:
df = pd.read_csv('./drive/My Drive/yelp_academic_dataset_review.csv', sep='\t',nrows=250000)
del df["Unnamed: 0"]
df

In [ ]:
#keep only the interested column
df2 = df[["stars", "text"]]
df2.head()

In [ ]:
# remove outliers 0 or 6 stars
df2 = df2.loc[(df2["stars"] < 6) and (df2["stars"] > 0)]
df2.head()
df2["stars"] = df2["stars"].astype(np.uint8)

In [ ]:
#divide the dataset into good and bad reviews
positiveReviews = df2.loc[df["stars"] > 3]
negativeReviews = df2.loc[df["stars"] < 3]
neutralReviews = df2.loc[df["stars"] == 3]
print(positiveReviews.shape)
print(negativeReviews.shape)
print(neutralReviews.shape)
dp = positiveReviews[:50000]
dn = negativeReviews[:50000]
#create a subset of 100000 reviews of the yelp dataset
df2 = dp.append(dn)
df2 = shuffle(df2)
df2.shape
df2.reset_index()


In [ ]:
#maximum and mean of word for each review
print(df2['text'].str.len().max())
print(df2['text'].str.len().mean())
df2.loc[df2['text'].str.len()==5000, 'text'].values

In [ ]:
#removing all not english reviews
df2['language'] = df2['text'].apply( lambda x: langid.classify(x)[0])
df2
not_english_reviews = df2.loc[df2['language']!= 'en']
not_english_reviews
df2 = df2.loc[df2['language'] == 'en']
not_english_reviews.shape


In [ ]:
#save the subset created
df2.to_csv('./drive/My Drive/subset_review.csv', sep='\t')


In [ ]:
#text cleaning using nltk
%%time
#tokenization, stop word removal and punctuation removal using nltk
#is slower than spacy
stop_words = stopwords.words("english")
stemmer = SnowballStemmer(language='english')

df2["text"] = df2["text"].str.lower()

#df2['cleaned_text_nltk'] = df2['text'].head(10000).apply(lambda x: [stemmer.stem(t.lower()) for t in normalise(word_tokenize(x)) if ( (not t in stop_words ) and (t.isalpha()))])
df2['cleaned_text_nltk'] = df2['text'].apply(lambda x: [stemmer.stem(t) for t in word_tokenize(x) if ( (not t in stop_words ) and (t.isalpha()))])

#df2['cleaned_text'] = df2['cleaned_text'].apply( [t.lemma.lemmatize(word, pos = "v") for t in df2['cleaned_text']  ])
#df2['cleaned_text'] = df2['cleaned_text'].apply( [t.lemma.lemmatize(word, pos = "n") for t in df2['cleaned_text']  ])
